In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import bgk

In [ ]:
h0 = 0.9
k = 0.1
B = 10

In [ ]:
input = bgk.Input(f"/mnt/lustre/IAM851/jm1667/psc/inputs/bgk/case1-B={B}-input.txt")
input.truncate(slice(0, 3000, 10))
beta = input.Te[0] ** .5
input.convert_to_cs_units()
T0 = input.Te[0]
print(f"T0={T0}")
print(f"beta={beta}")

## View f

In [ ]:
from math import exp, pi

def calc_v_phi(rho: float) -> float:
    return input.interpolate_value(rho, "v_phi")
def calc_psi(rho: float) -> float:
    return input.interpolate_value(rho, "Psi")
def calc_l(rho: float) -> float:
    return 2 * rho * calc_v_phi(rho) - B * rho**2
def calc_w(*, rho: float) -> float:
    return calc_v_phi(rho)**2 / 2 - calc_psi(rho)
def calc_f(*, rho: float=None, w: float=None, l: float=None) -> float:
    if rho is not None:
        w = calc_w(rho=rho)
        l = calc_l(rho=rho)
    return (2 * pi) ** (-3/2) * exp(-w/T0) * (1 - h0 *  exp(-k * l ** 2/T0))

In [ ]:
rhos = np.linspace(input.rho[0], input.rho[-2], 500)

ws = np.array([calc_w(rho=rho) for rho in rhos])
ls = np.array([calc_l(rho=rho) for rho in rhos])

fs1d = np.array([calc_f(w=w, l=l) for w, l in zip(ws, ls)])
fs2d = np.array([[calc_f(w=w, l=l) for w in ws] for l in ls])

maxf = np.max(fs2d)

In [ ]:
# f(w, l)
%matplotlib widget

plt.pcolormesh(*np.meshgrid(ws, ls), fs2d, cmap="Greens", vmax=maxf, shading="nearest")
plt.scatter(ws, ls, s=10., c=fs1d, cmap="Blues", vmax=maxf)
plt.colorbar()
plt.title(f"f(w,l) for B={B}")
plt.xlabel("w")
plt.ylabel("l");

In [ ]:
# w(rho)
%matplotlib widget
plt.plot(rhos, ws)

In [ ]:
# l(rho)
%matplotlib widget
plt.plot(rhos, ls)

In [ ]:
# f(v_phi, v_rho)
%matplotlib widget
rho = rhos[80]
psi = input.interpolate_value(rho, "Psi")

v_phis = np.linspace(-3, 3, 201)
v_rhos = np.linspace(-3, 3, 201)

fs2d_rho = np.array([[calc_f(w=(v_phi**2 + v_rho**2)/2 - psi, l=2*rho*v_phi - B * rho**2) for v_phi in v_phis] for v_rho in v_rhos])

plt.pcolormesh(*np.meshgrid(v_phis, v_rhos), fs2d_rho, cmap="Greens", shading="nearest")
plt.xlabel("v_phi")
plt.ylabel("v_rho")
plt.title(f"f(v_phi, v_rho) at rho={rho:.2f}")
plt.colorbar()

mean_v_phi = fs2d_rho.sum(axis=0).dot(v_phis) * (v_rhos[1] - v_rhos[0])
mean_v_rho = fs2d_rho.sum(axis=1).dot(v_rhos) * (v_phis[1] - v_phis[0])

print(f"mean v_phi: {mean_v_phi}")
print(f"mean v_rho: {mean_v_rho}")

In [ ]:
# f(v_phi) for fixed rho
%matplotlib widget
rho = rhos[50]
psi = input.interpolate_value(rho, "Psi")

v_phis = np.linspace(-3, 3, 201)
v_rhos = np.linspace(-3, 3, 201)
v_rho_integral = np.exp(-v_rhos ** 2 / 2).sum() * (v_rhos[1] - v_rhos[0])

# f(v_rho) is always a gaussian, so integrate it out and look at v_phi
fs1d_rho = v_rho_integral * np.array([calc_f(w=(v_phi**2)/2 - psi, l=2*rho*v_phi - B * rho**2) for v_phi in v_phis])

plt.plot(v_phis, fs1d_rho)
plt.xlabel("v_phi")
plt.ylabel("f")
plt.title(f"f(v_phi) at rho={rho:.2f}")

mean_v_phi = v_phis.dot(fs1d_rho)

print(f"mean v_phi: {mean_v_phi}")

In [ ]:
# f(rho, v_phi)
%matplotlib widget
v_phis = np.linspace(-3, 3, 201)
v_rhos = np.linspace(-3, 3, 201)
v_rho_integral = np.exp(-v_rhos ** 2 / 2).sum() * (v_rhos[1] - v_rhos[0])

# f(v_rho) is always a gaussian, so integrate it out and look at v_phi
fs2d_int = v_rho_integral * np.array([[calc_f(w=(v_phi**2)/2 - input.interpolate_value(rho, "Psi"), l=2*rho*v_phi - B * rho**2) for rho in rhos] for v_phi in v_phis])

# mean_v_phis_cs = np.array([calc_v_phi(rho) for rho in rhos])
mean_v_phis = fs2d_int.T.dot(v_phis) / fs2d_int.sum(axis=0) # agrees with just using calc_v_phi

plt.pcolormesh(*np.meshgrid(rhos * beta, v_phis * beta), fs2d_int, cmap="Reds", shading="nearest")
plt.plot(rhos*beta, mean_v_phis * beta, 'k')
# plt.plot(rhos*beta, mean_v_phis_cs * beta, 'r')
plt.xlabel("$\\rho$")
plt.ylabel("$v_\\phi$")
plt.title(f"f($\\rho$, $v_\\phi$) for $B={B}$")
plt.colorbar();